# Tutorial 4: Agents in LangChain

In this tutorial, we'll explore Agents in LangChain, which are autonomous entities capable of using tools and making decisions to accomplish tasks.

In [32]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.tools import Tool,DuckDuckGoSearchRun
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_sync_playwright_browser

# Load environment variables
load_dotenv()

# Initialize Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.7,
        model_kwargs={"top_p": 0.8, "seed": 1337}
    )

## 1. Understanding Agent Architecture

Agents in LangChain combine an LLM with a set of tools to perform tasks. The LLM decides which tools to use and how to interpret their outputs.

## 2. Exploring Different Types of Agents

### Zero-shot React Agent

In [3]:
search = DuckDuckGoSearchRun()

def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# tools = [
#     Tool(
#         name="DuckDuckGo Search",
#         func=search.run,
#         description="Useful for searching the internet for current information"
#     ),
#     Tool(
#     name="WordLength",
#     func=get_word_length,
#     description="Useful for getting the length of a word"
# )
# ]

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

zero_shot_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

zero_shot_agent.invoke(
    "What is the square root of the year Pythagoras was born?"
)

C:\Users\Domenico\AppData\Local\Temp\ipykernel_38232\400865729.py:22: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  zero_shot_agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: To find the square root of the year Pythagoras was born, I first need to find the year Pythagoras was born. 

Action: wikipedia
Action Input: Pythagoras birth year
Observation: Page: Pythagoras
Summary: Pythagoras of Samos (Ancient Greek: Πυθαγόρας; c. 570 – c. 495 BC) was an ancient Ionian Greek philosopher, polymath, and the eponymous founder of Pythagoreanism. His political and religious teachings were well known in Magna Graecia and influenced the philosophies of Plato, Aristotle, and, through them, the West in general. Knowledge of his life is clouded by legend; modern scholars disagree regarding Pythagoras's education and influences, but they do agree that, around 530 BC, he travelled to Croton in southern Italy, where he founded a school in which initiates were sworn to secrecy and lived a communal, ascetic lifestyle.
In antiquity, Pythagoras was credited with many mathematical and scientific discoveries, including the Pythagorean

{'input': 'What is the square root of the year Pythagoras was born?',
 'output': 'The square root of the year Pythagoras was born is approximately 23.87.'}

### Conversational Agent

In [12]:
conversational_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

conversational_agent.invoke({"input":"Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.","chat_history":""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: theory of relativity
Observation: Page: Theory of relativity
Summary: The theory of relativity usually encompasses two interrelated physics theories by Albert Einstein: special relativity and general relativity, proposed and published in 1905 and 1915, respectively. Special relativity applies to all physical phenomena in the absence of gravity. General relativity explains the law of gravitation and its relation to the forces of nature. It applies to the cosmological and astrophysical realm, including astronomy.
The theory transformed theoretical physics and astronomy during the 20th century, superseding a 200-year-old theory of mechanics created primarily by Isaac Newton. It introduced concepts including 4-dimensional spacetime as a unified entity of space and time, relativity of simultaneity, kinematic and gravitational time dilation, and length contraction. In the field of ph

{'input': "Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.",
 'chat_history': '',
 'output': "Now that we have a good understanding of the theory of relativity, what's the math problem you'd like to solve related to it?"}

### Plan-and-Execute Agent

In [15]:
plan_and_execute_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.PLAN_AND_EXECUTE,
    verbose=True
)

plan_and_execute_agent.run(
    "Find out who invented the telephone and calculate the number of years between that invention and the first moon landing."
)

AttributeError: PLAN_AND_EXECUTE

## 3. Creating Custom Tools for Agents

In [ ]:
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

word_length_tool = Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word"
)

custom_tools = [word_length_tool] + tools

custom_agent = initialize_agent(
    custom_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

custom_agent.run(
    "What's the length of the word 'supercalifragilisticexpialidocious' and its square root?"
)

## 4. Implementing a Multi-tool Agent for Task Solving

In [50]:
%autoawait asyncio

# Create a synchronous browser
sync_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(sync_browser=sync_browser)
browser_tools = browser_toolkit.get_tools()

# Combine all tools
all_tools = tools + browser_tools + [word_length_tool]

multi_tool_agent = initialize_agent(
    all_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

multi_tool_agent.run(
    """Please perform the following tasks:
    1. Find the current temperature in New York City.
    2. Calculate the square root of that temperature.
    3. Find a famous quote about weather and tell me how many characters it has.
    Provide a summary of your findings.
    """
)

# Close the browser
sync_browser.close()

RuntimeError: This event loop is already running

## Conclusion

In this tutorial, we've explored Agents in LangChain, including different types of agents, creating custom tools, and implementing a multi-tool agent for complex task solving. Agents provide a powerful way to create autonomous systems that can leverage various tools and make decisions to accomplish tasks.